In [1]:
import requests
import pandas as pd
import os
import wget

In [2]:
root_folder = 'dados/'

folder_renach = 'renach/'
folder_infracoes = 'infracoes/'

#### Lendo dados dos arquivos do RENACH

In [ ]:
os.chdir(root_folder + folder_renach)

for ano in [2018, 2019, 2020, 2021, 2022]:
    for mes in range(1, 13):    
        
        try:
            data = f'{ano}_{mes:02d}'
            url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renach/csv/condutores_habilitados_{data}.csv'
            
            wget.download(url)
        except:
            data = f'{ano}-{mes:02d}'
            url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renach/csv/condutores_habilitados_{data}.csv'

            wget.download(url)

os.chdir('../..')            

In [ ]:
df_result = pd.DataFrame()

for file in os.listdir(root_folder + folder_renach):
    df = pd.read_csv(root_folder + folder_renach + file, encoding='utf_16', decimal=',', thousands='.')

    file = file.replace('-', '_')  
    df['ano_mes'] = int(file.split('_')[2] + file.split('_')[3].split('.')[0])
    df_result = pd.concat([df_result, df], axis='index')

## Adiquirindo a relação de nome/sigla das UFs

In [ ]:
response = requests.get('http://servicodados.ibge.gov.br/api/v1/localidades/estados?orderBy=id').json()

In [ ]:
ufs = {}
for uf in response: ufs[uf['nome']] = uf['sigla']

In [ ]:
df_result['uf'] = [ufs[nome_uf] for nome_uf in df_result['UF Habilitação Atual']]
df_result['categoria_cnh'] = df_result['Categoria'].str.split('-', expand=True)[0]

In [ ]:
df_result = df_result[['uf', 'Sexo', 'Faixa Etária', 'categoria_cnh', 'Qt. Condutor Histórico', 'ano_mes']]

quantidade_condutores = df_result.rename(columns={
    'Sexo':'sexo',
    'Faixa Etária':'faixa_etaria',
    'Qt. Condutor Histórico':'qtd_condutores'
})

In [ ]:
quantidade_condutores

## Adiquirindo dados sobre os tipos de infrações

In [ ]:
os.chdir(root_folder + folder_infracoes)

try:
    url = 'https://www.gov.br/transportes/pt-br/centrais-de-conteudo/tabela-codigo-infracoes-renainf-xlsx'
    
    wget.download(url)
except:
    print('Erro de download.')

os.chdir('../..')  

In [ ]:
df_infracoes = pd.read_excel(root_folder + folder_infracoes + 'teste/tabela-codigo-infracoes-renainf-xlsx', engine='openpyxl')

In [ ]:
df_infracoes['id'] = df_infracoes.index + 1
df_infracoes = df_infracoes[['id', 'Código da Infração', 'Descrição da Infração', 'Gravidade', 'Órgão Competente']]

infracoes = df_infracoes.rename(columns={
    'Código da Infração':'codigo_infracao',
    'Descrição da Infração':'descricao_infracao',
    'Gravidade':'gravidade',
    'Órgão Competente':'orgao_competente'
})

In [ ]:
infracoes

## Adiquirindo dados do RENAEST

#### Localidade

In [ ]:
df_localidade = pd.read_csv('dados/renaest/Localidade_DadosAbertos_20230912.csv', sep=';')

In [ ]:
df_localidade.head()

In [ ]:
df_localidade.drop_duplicates(subset=['mes_ano_referencia', 'codigo_ibge'], inplace=True)
df_localidade = df_localidade[df_localidade['ano_referencia'] < 2023]

df_localidade['ano_mes'] = df_localidade['ano_referencia'] * 100 + df_localidade['mes_referencia']
df_localidade['metropolitana'] = df_localidade['regiao_metropolitana'] == 'sim'

df_localidade = df_localidade.rename(columns={
    'qtde_habitantes':'qtd_habitantes'
})

In [ ]:
localidade = df_localidade[['ano_mes', 'municipio', 'uf', 'metropolitana', 'qtd_habitantes', 'frota_total', 'frota_circulante', 'regiao', 'codigo_ibge']].reset_index(drop=True)
localidade['id'] = localidade.index + 1

In [ ]:
localidade

#### Acidentes

In [3]:
df_acidentes = pd.read_csv('dados/renaest/Acidentes_DadosAbertos_20230912.csv', sep=';')

/tmp/ipykernel_5482/1315882353.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_acidentes = pd.read_csv('dados/renaest/Acidentes_DadosAbertos_20230912.csv', sep=';')


In [4]:
df_acidentes.columns

Index(['num_acidente', 'chv_localidade', 'data_acidente', 'uf_acidente',
       'ano_acidente', 'mes_acidente', 'mes_ano_acidente', 'codigo_ibge',
       'dia_semana', 'fase_dia', 'tp_acidente', 'cond_meteorologica',
       'end_acidente', 'num_end_acidente', 'cep_acidente', 'bairro_acidente',
       'km_via_acidente', 'latitude_acidente', 'longitude_acidente',
       'hora_acidente', 'tp_rodovia', 'cond_pista', 'tp_cruzamento',
       'tp_pavimento', 'tp_curva', 'lim_velocidade', 'tp_pista',
       'ind_guardrail', 'ind_cantcentral', 'ind_acostamento', 'qtde_acidente',
       'qtde_acid_com_obitos', 'qtde_envolvidos', 'qtde_feridosilesos',
       'qtde_obitos'],
      dtype='object')

#### Veículo

In [11]:
df_veiculo = pd.read_csv('dados/renaest/TipoVeiculo_DadosAbertos_20230912.csv', sep=';')

In [15]:
df_veiculo.drop_duplicates(subset=['tipo_veiculo'], inplace=True)
df_veiculo = df_veiculo[['tipo_veiculo']].reset_index(drop=True)
df_veiculo['id'] = df_veiculo.index + 1

In [16]:
veiculo = df_veiculo.rename(columns={
    'tipo_veiculo':'tipo'
})

In [17]:
veiculo

,tipo,id
0,NAO INFORMADO,1
1,AUTOMOVEL,2
2,BICICLETA,3
3,BONDE,4
4,CAMINHAO,5
5,CAMINHAO-TRATOR,6
6,CAMINHONETE,7
7,CAMIONETA,8
8,CARRO DE MAO,9
9,CARROCA,10


## Adiquirindo os dados de quantidades de infrações

In [ ]:
os.chdir(root_folder + folder_infracoes)

for ano in [2019, 2020, 2021, 2022]:
    for mes in range(1, 13):    
        
        try:
            data = f'{ano}_{mes:02d}'
            url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renainf/csv/{data}_infracoes_com_np.csv'
            
            wget.download(url)
        except:
            data = f'{ano}-{mes:02d}'
            url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renainf/csv/{data}_infracoes_com_np.csv'

            wget.download(url)

os.chdir('../..')            

In [ ]:
df_result = pd.DataFrame()

for file in os.listdir(root_folder + folder_infracoes):
    df = pd.read_csv(root_folder + folder_infracoes + file, sep=';', encoding='ISO-8859-1')

    file = file.replace('-', '_')  
    print(file)
    df['ano_mes'] = int(file.split('_')[0] + file.split('_')[1])
    df_result = pd.concat([df_result, df], axis='index')

In [ ]:
df = pd.read_excel('/home/lucas/Downloads/2022_09_infracoes_com_np.xlsx', engine='openpyxl')

In [ ]:
df

In [ ]:
localidade_url = 'http://servicodados.ibge.gov.br/api/v1/localidades/municipios'

response = requests.get(url=localidade_url).json()

In [ ]:
response[0]